In [1]:
import numpy as np
import math
import random
import os
import pybullet as p
import pybullet_data
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import matplotlib.pyplot as plt
import time
import cv2

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Dropout, concatenate
from keras.layers import Concatenate,Conv2D,BatchNormalization,MaxPooling2D
from keras.optimizers import Adam

C:\Users\SERVER\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framew

In [3]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import  SequentialMemory
from rl.processors import Processor

In [4]:
class BikeEnv(gym.Env):
    
    def __init__(self):
        p.connect(p.GUI)
        p.setRealTimeSimulation(1)
        p.resetDebugVisualizerCamera(cameraDistance=10, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55,-0.35,0.2])
        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.Box(low=0, high=1, shape=(64, 64))
        self.timestep = 1./240.
        self.stp = 0
        
    def step(self, action):
        print("i=%d" % self.stp)
        self.stp+=1
        #print("time = " , time.time()-self.time)
        self.time =time.time()
        if (action == 0):
            self.speed = self.speed + 1
        if (action == 1):
            self.speed = self.speed - 1 
        if (action == 2):
            self.speed = self.speed  
        if (action == 3):
            self.steer = self.steer - 1 
        if (action == 4):
            self.steer = self.steer + 1
        if (action == 5):
            self.steer = self.steer 
            
              
        self.applyAction([self.speed,self.steer])
        time.sleep(0.2)
        
        state = p.getLinkState(self.pid,0)[0]
        if state[2] <= 0.5 or  state[2] >= 2 or abs(self.speed)>2 or abs(self.steer)>4:
            reward = -100
            done = True
            print('done!')
        else :
            #reward = math.sqrt((self.origin[0]-state[0])**2+(self.origin[1]-state[1])**2)
            reward = state[0] - self.origin[0]
            #reward = 1
            done = False
        self.origin = state 
        
        velocity = p.getBaseVelocity(self.pid)
        img = self.getImage()
        observation = img
        info = {'x':'','y':'','z':''}
        #print("Step: ",self.stp)
        #xx = time.time()
        #print("Time: ",xx-self.tttt)
        #self.tttt = xx
        #print("Action: ",action)
        #print("Reward: ",reward)
        #self.stp +=1
        return observation, reward, done, info
            
    def applyAction(self, motorCommands):
        targetVelocity = motorCommands[0] * self.speedMultiplier
        #print("targetVelocity")
        #print(targetVelocity)
        steeringAngle = motorCommands[1] * self.steeringMultiplier
        #print("steeringAngle")
        #print(steeringAngle)


        for motor in self.motorizedwheels:
            p.setJointMotorControl2(self.pid,
                                    motor,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=targetVelocity,
                                    force=self.maxForce)
        for steer in self.steeringLinks:
            p.setJointMotorControl2(self.pid,
                                    steer,
                                    p.POSITION_CONTROL,
                                    targetPosition=steeringAngle)

    def reset(self):
        print("Reset")
        self.stp = 0
        self.time =time.time()
        #print("===========Reset=====")
        p.resetSimulation()

        urdfRootPath = pybullet_data.getDataPath()
        planeUid = p.loadURDF(os.path.join(urdfRootPath,"plane.urdf"), basePosition=[0,0,0])
        
        
        for i in range(np.random.randint(1,10)):
            p.loadURDF(os.path.join(urdfRootPath, "sphere2.urdf"),basePosition=[
                np.random.randint(5,15),
                np.random.randint(-2,2),
                0.5
            ])
        time.sleep(1)     
        self.pid = p.loadURDF(os.path.join(urdfRootPath, "bicycle/bike.urdf"),basePosition=[0,0,1])     
           
        
        self.origin = p.getLinkState(self.pid,0)[0]
        p.setGravity(0,0,-10)
        for wheel in range(p.getNumJoints(self.pid)):
            p.setJointMotorControl2(self.pid,
                                    wheel,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=0,
                                    force=0)

        self.steeringLinks = [0]
        self.maxForce = 20
        self.nMotors = 2
        self.motorizedwheels = [1, 2]
        self.speedMultiplier = 10.
        self.steeringMultiplier = 0.5
        
        self.speed = 0 
        self.steer = 0

        velocity = p.getBaseVelocity(self.pid)
        img = self.getImage()
        observation = img
        p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
        
        return observation
        
    
    def getImage(self):
        img = env.render()
        img = cv2.resize(img, (64,64), interpolation = cv2.INTER_NEAREST )
        img = img[:,:,2]
        img =img.reshape((64,64))
        img = np.asarray(img, dtype='float32')
        img /= 255.0
        return img
        
    def getObservationDimension(self):
        return len(self.getObservation())
    
    def getObservation(self):
        observation = []
        pos, orn = p.getBasePositionAndOrientation(self.pid)

        #observation.extend(list(pos))
        observation.extend(list(orn))
        return observation
        
    def render(self, mode='human'):
        pos, orn = p.getBasePositionAndOrientation(self.pid)
      
        view_matrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[pos[0]+11.3,pos[1],pos[2]],
                                                            distance=10,
                                                            yaw=-90 ,
                                                            pitch=0,
                                                            roll=0,
                                                            upAxisIndex=2)
        proj_matrix = p.computeProjectionMatrixFOV(fov=60,
                                                     aspect=float(960) /720,
                                                     nearVal=0.1,
                                                     farVal=100.0)
        (_, _, px, _, _) = p.getCameraImage(width=960,
                                              height=720,
                                              viewMatrix=view_matrix,
                                              projectionMatrix=proj_matrix,
                                              renderer=p.ER_BULLET_HARDWARE_OPENGL)

        rgb_array = np.array(px, dtype=np.uint8)
        rgb_array = np.reshape(rgb_array, (720,960, 4))

        rgb_array = rgb_array[:, :, :3]
        return rgb_array

    def close(self):
        p.disconnect()

In [5]:
env = BikeEnv()
np.random.seed(123)
env.seed(123)

In [6]:
nb_actions = env.action_space.n

In [7]:
def create_mlp():
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + env.observation_space['sensor'].shape))
    model.add(Dense(16))
    model.add(Activation('relu'))
    model.add(Dense(16))
    model.add(Activation('relu'))
    model.add(Dense(16))
    model.add(Activation('relu'))
    return model

In [8]:
def create_cnn():
    inputShape = (64, 64,1)
    inputs = Input(shape=inputShape,name='image')
    x = Conv2D(16, (3, 3), padding="same")(inputs)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(64, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(128, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(6)(x)
    x = Activation("linear")(x)
    
    model = Model(inputs, x)
    return model

In [9]:
#mlp = create_mlp()
cnn = create_cnn()
#combinedInput = concatenate([mlp.output, cnn.output])
#x = Dense(nb_actions, activation="relu")(combinedInput)
#model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [10]:
class CustomProcessor(Processor):

    def process_state_batch(self, batch):
        '''
        Given a state batch, I want to remove the second dimension, because it's
        useless and prevents me from feeding the tensor into my CNN
        '''
        batch = np.squeeze(batch, axis=1)
        return batch
    def process_observation(self, observation):
        observation = observation.reshape((64,64,1))
        return observation

In [11]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=cnn, nb_actions=nb_actions, memory=memory, nb_steps_warmup=100,
               target_model_update=1e-2, policy=policy)
dqn.processor = CustomProcessor()
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [12]:
dqn.fit(env, nb_steps=100000, visualize=False, verbose=1)

Training for 100000 steps ...
Reset
getimage= 0.171600341796875
Interval 1 (0 steps performed)

i=0
getimage= 0.10920000076293945
    1/10000 [..............................] - ETA: 2:22:59 - reward: 0.0808i=1
getimage= 0.09360027313232422
    2/10000 [..............................] - ETA: 1:40:04 - reward: 0.2292i=2
getimage= 0.10920047760009766
    3/10000 [..............................] - ETA: 1:25:46 - reward: 0.3135i=3
done!
getimage= 0.10920000076293945
    4/10000 [..............................] - ETA: 1:18:37 - reward: -24.7649Reset
getimage= 0.12480020523071289
i=0
getimage= 0.09360027313232422
    5/10000 [..............................] - ETA: 2:16:10 - reward: -19.8125i=1
getimage= 0.10920000076293945
    6/10000 [..............................] - ETA: 2:02:59 - reward: -16.5097i=2
getimage= 0.09359979629516602
    7/10000 [..............................] - ETA: 1:52:50 - reward: -14.1505i=3
getimage= 0.10920000076293945
    8/10000 [..............................] - ETA

   67/10000 [..............................] - ETA: 1:48:14 - reward: -16.4146i=2
getimage= 0.09360027313232422
   68/10000 [..............................] - ETA: 1:47:23 - reward: -16.1628i=3
getimage= 0.09360027313232422
   69/10000 [..............................] - ETA: 1:46:34 - reward: -15.9156i=4
getimage= 0.09360003471374512
   70/10000 [..............................] - ETA: 1:45:49 - reward: -15.6742i=5
done!
getimage= 0.09359979629516602
   71/10000 [..............................] - ETA: 1:45:02 - reward: -16.8619Reset
getimage= 0.12480020523071289
i=0
getimage= 0.12480020523071289
   72/10000 [..............................] - ETA: 1:48:50 - reward: -16.6266i=1
getimage= 0.10920000076293945
   73/10000 [..............................] - ETA: 1:48:07 - reward: -16.3928i=2
getimage= 0.10920047760009766
   74/10000 [..............................] - ETA: 1:47:27 - reward: -16.1623i=3
done!
getimage= 0.10920047760009766
   75/10000 [..............................] - ETA: 1:46

getimage= 0.12480020523071289
  133/10000 [..............................] - ETA: 3:10:58 - reward: -23.3500i=1
done!
getimage= 0.09360027313232422
  134/10000 [..............................] - ETA: 3:11:10 - reward: -23.9221Reset
getimage= 0.06240034103393555
i=0
getimage= 0.07800006866455078
  135/10000 [..............................] - ETA: 3:14:44 - reward: -23.7456i=1
done!
getimage= 0.07800030708312988
  136/10000 [..............................] - ETA: 3:14:41 - reward: -24.3063Reset
getimage= 0.09360003471374512
i=0
getimage= 0.07800006866455078
  137/10000 [..............................] - ETA: 3:18:19 - reward: -24.1295i=1
done!
getimage= 0.062399864196777344
  138/10000 [..............................] - ETA: 3:18:28 - reward: -24.6793Reset
getimage= 0.062400102615356445
i=0
getimage= 0.04680013656616211
  139/10000 [..............................] - ETA: 3:21:38 - reward: -24.5012i=1
done!
getimage= 0.031199932098388672
  140/10000 [..............................] - ETA: